In [ ]:
import sqlite3 as sql
import pandas as pd
import os
import matplotlib.pyplot as plt
import functools
import re
from matplotlib import pyplot as plt
import numpy as np
from sklearn import metrics as sk_metrics
%matplotlib inline

In [817]:
# location of trends.csv
data_dir='/home/matt/Datasets/Battelle Data/data-extracted/'
# location of databases
db_dir='/home/matt/Datasets/Battelle Data/databases/'
main_database = 'battelle.db'
# save results of model testing here
results_output='testing_results.csv'

## read the labels

In [316]:
os.chdir(data_dir)
words = pd.read_csv("trends.csv")
words.columns = ['word_str','label']
words.set_index('word_str',inplace=True)

## 0-1 encode the labels

In [317]:
def zero_one(x):
    if x == 100:
        return 1
    return 0

In [318]:
words.label = words.label.apply(zero_one)

In [319]:
words.head()

,label
word_str,
ability,0
abiotic,1
absorption,0
abundance,1
abundant,1


## get the word counts from the db

In [795]:
os.chdir(db_dir)
con = sql.connect(main_database)
cur = con.cursor()

In [217]:
command = '''
SELECT d.doc_label as year,
       word_id,
       word_str,
       sum(word_count) as word_count
  FROM docword w
       JOIN
       doc d USING (
           doc_id
       )
  WHERE word_str in {}
 GROUP BY doc_label,word_id'''


In [218]:
# selector list for sql query: ('word1','word2', ...)
id_string ='('+re.sub("'",'"',re.sub('[\[\]]','',str(list(words.title))))+')'

In [219]:
word_counts = pd.read_sql_query(command.format(id_string),con)

In [220]:
# sanity check
word_counts.columns
word_counts.shape

(10957, 4)

In [221]:
# don't need the word ids
word_counts.drop('word_id',axis=1,inplace=True)
# reindex and reshape
word_counts.set_index(['word_str','year'],inplace=True)
word_counts=word_counts.unstack(level=1)

In [222]:
word_counts.head()

word_count                                                  
year             2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
word_str                                                               
ability            68  126  159  213  341  406  458  462  592  659  300
abiotic            12   14   18   17   25   49   57  101  133  171  109
absorption         31   35   47   74  140  130  154  163  167  203   76
abundance          25   45   63   60   93  111  149  185  252  286  192
abundant           14   34   36   46   82   97  126  150  196  223  136

## get total term frequency by year

In [148]:
command = """
SELECT SUM(word_count), doc_label
FROM docword
JOIN doc USING (doc_id)
GROUP BY doc_label
"""
ttf = pd.read_sql_query(command,con)

In [155]:
ttf.columns = ['count','year']
ttf.set_index('year',inplace=True)

In [161]:
# make a pandas Series
ttf = ttf['count']

In [206]:
# sanity check on unigenes
test = word_counts2.loc[cur.execute("select word_id from docword where word_str = 'unigenes'").fetchone()[0]]/ttf
plt.bar(left=range(2005,2016),height=test)

## normalize frequencies to proportions

In [229]:
word_props = word_counts.apply(lambda x: x/ttf,axis=1)
word_props.fillna(0,inplace=True)

In [ ]:
# sanity check
plt.bar(word_props.loc[0])

## normalize series to shapes

In [244]:
trends = word_props.apply(lambda x: x/x.sum(),axis=1)

In [254]:
trends.head()

word_count                                                    \
year             2005      2006      2007      2008      2009      2010   
word_str                                                                  
ability      0.073113  0.095966  0.096506  0.094665  0.096796  0.096094   
abiotic      0.077529  0.064073  0.065649  0.045400  0.042642  0.069690   
absorption   0.101294  0.081012  0.086694  0.099949  0.120773  0.093509   
abundance    0.071390  0.091027  0.101557  0.070823  0.070113  0.069776   
abundant     0.053228  0.091570  0.077265  0.072293  0.082308  0.081184   

                                                              
year            2011      2012      2013      2014      2015  
word_str                                                      
ability     0.091304  0.087554  0.091251  0.091918  0.084832  
abiotic     0.068281  0.115015  0.123188  0.143322  0.185211  
absorption  0.093301  0.093877  0.078229  0.086050  0.065312  
abundance   0.078890  0.093115  0.103164  0.105948  0.144196  
abundant    0.088823  0.100520  0.106831  0.109989  0.135990

In [255]:
words.head()

,label
word_str,
ability,50
abiotic,100
absorption,50
abundance,100
abundant,100


In [264]:
words.sort_index(inplace=True)
trends.sort_index(inplace=True)

## the model

In [482]:
def laplace_kernel(v1,v2,gamma):
    return np.exp(-1*gamma*np.sqrt(np.sum(np.square(v1-v2))))
def gaussian_kernel(v1,v2,gamma):
    return np.exp(-1*gamma*(np.sum(np.square(v1-v2))))

In [449]:
def classify(sequence,labels,trends,gamma,kernel):
    scores = trends.apply(lambda v:kernel(np.array(v),np.array(sequence),gamma),axis=1)
    pos = labels*scores
    neg = (1-labels)*scores
    return pos.mean()/neg.mean()

## An example

In [678]:
# a positive case
i=92
print(words.label.iloc[i])
classify(trends.iloc[i],words.label,trends,gamma=50,kernel=laplace_kernel)

1


1.3119307852120927

In [625]:
# a negative case
i=100
print(words.label.iloc[i])
classify(trends.iloc[i],words.label,trends,gamma=50,kernel=laplace_kernel)

0


0.20647842077542944

## validate parameters

In [844]:
# all the parameters to iterate over
metrics = ['precision','recall','F1','AUC']
kernels = ['laplace','gaussian']
gammas = list(np.arange(3,33)*10)
results_index = product(kernels,gammas,metrics)

In [845]:
# build a results df from these parameters
results = pd.DataFrame(index=pd.MultiIndex.from_product([kernels,gammas,metrics],
                                                        names=['kernels','gammas','metrics']),columns=range(10))

In [846]:
# maps from names to functions
kernel_map = {'laplace':laplace_kernel,'gaussian':gaussian_kernel}
metric_map = {'precision':sk_metrics.precision_score,
              'recall':sk_metrics.recall_score,
              'F1':sk_metrics.f1_score,
              'AUC':sk_metrics.roc_auc_score}

In [ ]:
# for every combination of parameters,
for params in results.index:
    kernel,gamma,metric=params
    print()
    print(params)
    
    # and all 10 trials,
    for trial in results.columns:
        # split into training and testing sets 70/30
        train_X = trends.sample(frac=0.7)
        train_y = words.loc[train_X.index]
        test_index = trends.index.difference(train_X.index)
        test_X = trends.loc[test_index]
        test_y = words.loc[test_index]
        # compute the score for every test sequence
        trendiness = test_X.apply(lambda x: classify(x,labels=train_y.label,
                                                     trends=train_X,
                                                     gamma=gamma,
                                                     kernel=kernel_map[kernel]),
                                                     axis=1)
        # convert to binary response if not evaluating using AUC
        if metric != 'AUC':
            trendiness = (trendiness > 1).astype('int')
        # compute the accuracy metric
        accuracy = metric_map[metric](test_y,trendiness)
        print(accuracy,end='\t')
            
        results.loc[params,trial] = accuracy


('laplace', 30, 'precision')
0.875	1.0	1.0	0.875	0.875	0.875	1.0	1.0	1.0	1.0	
('laplace', 30, 'recall')
0.135593220339	0.15	0.131147540984	0.109375	0.224489795918	0.129032258065	0.0727272727273	0.10447761194

In [ ]:
results.to_csv(data_dir+results_output)

## get series for every word

In [400]:
# get series for every word
command = '''
SELECT d.doc_label as year,
       word_id,
       word_str,
       sum(word_count) as word_count
  FROM docword w
       JOIN
       doc d USING (
           doc_id
       )
 GROUP BY word_id, d.doc_label'''

In [401]:
word_series = pd.read_sql_query(command,con)

In [402]:
# reindex and reshape
word_series.set_index(['word_id','word_str','year'],inplace=True)
word_series=word_series.unstack(level='year')

word_series.fillna(0,inplace=True)
word_series.columns = word_series.columns.get_level_values(1)

In [531]:
word_series.head()

,year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
word_id,word_str,,,,,,,,,,,
0,Relationship,0,0,0,0,0,1,3,1,0,0,0
1,IGFBP7,0,0,1,0,0,10,1,9,1,0,0
2,cross-layer,1,0,3,4,4,3,1,6,1,0,0
3,instability,1,12,7,27,17,46,41,20,47,59,29
4,prerequisite,4,3,5,11,8,20,15,17,26,16,4


In [532]:
# normalize to proportions
word_props = word_series.apply(lambda x: x/ttf,axis=1)
word_props.fillna(0,inplace=True)

In [533]:
# normalize from series to shapes
word_trends = word_props.apply(lambda x: x/x.sum(),axis=1)

In [534]:
word_trends.head()

,year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
word_id,word_str,,,,,,,,,,,
0,Relationship,0.000000,0.000000,0.000000,0.000000,0.000000,0.231080,0.583898,0.185022,0.000000,0.000000,0.000000
1,IGFBP7,0.000000,0.000000,0.120597,0.000000,0.000000,0.470277,0.039610,0.338890,0.030626,0.000000,0.000000
2,cross-layer,0.134234,0.000000,0.227327,0.221945,0.141755,0.088648,0.024889,0.141958,0.019244,0.000000,0.000000
3,instability,0.013817,0.117454,0.054600,0.154210,0.062014,0.139916,0.105039,0.048708,0.093101,0.105757,0.105385
4,prerequisite,0.122542,0.065104,0.086470,0.139297,0.064704,0.134878,0.085203,0.091796,0.114190,0.063588,0.032228


## compute trendiness for every word

In [717]:
# new dataframe to hold stats; this gets written to a db
word_stats = pd.DataFrame([t for t in word_series.index],columns=['word_id','word_str'])
word_stats.set_index(['word_id'],inplace=True)

In [718]:
word_stats.head()

,word_str
word_id,
0,Relationship
1,IGFBP7
2,cross-layer
3,instability
4,prerequisite


In [708]:
# timing the classification
%timeit trendiness = word_trends.iloc[0:100].apply(classify,axis=1,**dict(labels=words.label,trends=trends,gamma=50,kernel=laplace_kernel))
# at this rate it should take < 3.22s*44000/(100*60) ~ 23 mins to complete

1 loops, best of 3: 3.12 s per loop


In [457]:
word_stats.trendiness = word_trends.iloc.apply(classify,axis=1,**dict(
                labels=words.label,trends=trends,gamma=50,kernel=laplace_kernel))

(44326, 11)

### Questions:
    -Does another kernel perform better?
    -How are the results affected by gamma?
    -ROC curves for varying gamma?
    -Does it make sense to transform the data first?
        *smoothing: rolling average
        *log or power transform: most kernels make more sense in this transformed space

## Compute other relevant stats:
    -ttf
    -df
    -idf

In [719]:
# ttf
command="""
SELECT word_id,
       word_str,
       word_freq
FROM word
"""

In [726]:
#word_stats.ttf = 
ttf_all = pd.read_sql_query(command,con)
ttf_all.set_index('word_id',inplace=True)

In [730]:
word_stats.loc[:,'ttf'] = ttf_all.word_freq

In [731]:
word_stats.head()

,word_str,ttf
word_id,,
0,Relationship,5
1,IGFBP7,22
2,cross-layer,23
3,instability,306
4,prerequisite,129


In [776]:
# df
command="""
SELECT word_id, count(doc_id) as df
FROM docword
 GROUP BY word_id
"""

In [777]:
df_all = pd.read_sql_query(command,con)
df_all.set_index('word_id',inplace=True)

In [803]:
df_all.head()

,df
word_id,
0,5
1,5
2,17
3,238
4,129


In [779]:
word_stats.loc[:,'df'] = df_all.df

In [781]:
command="""
SELECT count(doc_id) 
FROM doc"""
doc_count = cur.execute(command).fetchall()

doc_count = doc_count[0][0]

In [786]:
#idf
word_stats.loc[:,'idf'] = doc_count/df_all.df
word_stats.loc[:,'idf'] = word_stats.idf.apply(np.log)

In [788]:
word_stats.head()

,word_str,ttf,df,idf
word_id,,,,
0,Relationship,5,5,9.397567
1,IGFBP7,22,5,9.397567
2,cross-layer,23,17,8.173791
3,instability,306,238,5.534734
4,prerequisite,129,129,6.147192


In [800]:
## write the stats to a db
con2 = sql.connect("word_stats.db")
word_stats.to_sql("word_stats",con2,schema='''("word_id" INTEGER PRIMARY KEY, "word_str" TEXT,
  "ttf" INTEGER,
  "df" INTEGER,
  "idf" REAL)''')

In [801]:
con2.close()

## That's all, folks!